In [59]:
import pandas as pd

df = pd.read_csv("../data/spi_matches.csv", parse_dates=["date"])

df = df[df.league_id == 2411]

df = df[df.score1.isnull()]

df.head()
df = df[(df.date < pd.datetime(2019, 4, 7)) & (df.date > pd.datetime(2019, 1, 28))]

In [60]:
max_per_team = df.groupby(["team1"]).agg("max")["prob1"].reset_index().sort_values("prob1", ascending=False).reset_index(drop=True)
max_per_team

,team1,prob1
0,Manchester City,0.8998
1,Liverpool,0.8869
2,Chelsea,0.7875
3,Tottenham Hotspur,0.7640
4,Arsenal,0.7065
5,Manchester United,0.6793
6,Watford,0.5832
7,Leicester City,0.5575
8,Crystal Palace,0.5527
9,Southampton,0.5404


In [61]:
max_per_team.iloc[6:12]

,team1,prob1
6,Watford,0.5832
7,Leicester City,0.5575
8,Crystal Palace,0.5527
9,Southampton,0.5404
10,Wolverhampton,0.5395
11,West Ham United,0.5374


In [66]:
picks = pd.merge(df, max_per_team.iloc[6:12], how="inner", on=["team1", "prob1"])[["date", "team1", "team2", "prob1"]]

In [62]:
picks = pd.merge(df, max_per_team.iloc[6:12], how="inner", on=["team1", "prob1"])[["date", "team1", "team2", "prob1"]]

In [63]:
picks.sort_values("date")

,date,team1,team2,prob1
0,2019-02-09,Southampton,Cardiff City,0.5404
1,2019-03-02,Wolverhampton,Cardiff City,0.5395
2,2019-03-09,Leicester City,Fulham,0.5575
3,2019-03-16,West Ham United,Huddersfield Town,0.5374
4,2019-03-30,Crystal Palace,Huddersfield Town,0.5527
5,2019-04-06,Watford,Fulham,0.5832


In [68]:
pd.merge(df, max_per_team.iloc[6:10], how="inner", on=["team1", "prob1"])[["date", "team1", "team2", "prob1"]]

,date,team1,team2,prob1
0,2019-02-09,Southampton,Cardiff City,0.5404
1,2019-03-09,Leicester City,Fulham,0.5575
2,2019-03-30,Crystal Palace,Huddersfield Town,0.5527
3,2019-04-06,Watford,Fulham,0.5832
